# skips

In [1]:
from random import sample, choice, randint
from avrasm import *

In [2]:
def setreg(reg, val):
    return LDS(r(reg), a(0)).read(val, 0)(f"set r{reg} to {val}")

def checkreg(reg, val):
    return STS(a(0), r(reg)).write(val, 0)(f"r{reg} should be {val}")

def setstatus(bit, isSet=True):
    if isSet:
        return BSET(bit)
    else:
        return BCLR(bit)

def checkstatus(bit, isSet=True, label="done"):
    mssg = f"bit {bit} of status should be {'set' if isSet else 'clear'}"
    if isSet:
        return BRBC(bit, label)(mssg)
    else:
        return BRBS(bit, label)(mssg)

In [3]:
def testCpse(regs, vals, instr):
    """test CPSE by setting two registers to values,
    and seeing if instruction i is skipped
    """
    reg1, reg2 = regs
    val1, val2 = vals
    skip = val1 == val2
    mssg = f"{val1} {'==' if skip else '!='} {val2}"
    if (skip):
        instr.skip()
    return [
        setreg(reg1, val1),
        setreg(reg2, val2),
        CPSE(r(reg1), r(reg2)),
        instr(mssg)
    ]

In [4]:
cpseTests = [
    "\n;PREPROCESS TestCPSE",
    LDI(r(31), v(0x00)),
    LDI(r(30), v(0x00)),
    "; skip a LD (1 word) instruction",
    testCpse(sample(range(30), 2), (0, 0), LD(r(0), 'Z').read(0, 0)),
    "; don't skip a LD instruction",
    testCpse(sample(range(30), 2), (0, randint(0,0xFF)), LD(r(0), 'Z').read(0, 0)),
    "; skip a STS (2 word) instruction",
    testCpse(sample(range(32), 2), (0, 0), checkreg(0,0xff)),
    "; don't skip a STS instruction",
    testCpse(sample(range(32), 2), (randint(0,0xFE), 0xFF), checkreg(0,0xff)),
]

In [5]:
def testSbrc(reg, val, bit, instr):
    skip = not(val & (1<<bit))
    mssg = f"bit {bit} of {val:08b} is {'clear' if skip else 'set'}"
    if skip:
        instr.skip()
    return [
        setreg(reg, val),
        SBRC(r(reg), str(bit)),
        instr(mssg)
    ]

In [6]:
sbrcTests = [
    "\n;PREPROCESS TestSBRC",
    LDI(r(31), v(0x00)),
    LDI(r(30), v(0x00)),
    "; skip a LD (1 word) instruction",
    testSbrc(choice(range(30)), 0xF0, 0, LD(r(0), 'Z').read(0, 0)),
    "; don't skip a LD instruction",
    testSbrc(choice(range(30)), 0x0F, 3, LD(r(0), 'Z').read(0, 0)),
    "; skip a STS (2 word) instruction",
    testSbrc(choice(range(32)), 0xF0, 1, checkreg(0,0xff)),
    "; don't skip a STS instruction",
    testSbrc(choice(range(32)), 0x0F, 2, checkreg(0,0xff)),
]

In [7]:
def testSbrS(reg, val, bit, instr):
    skip = (val & (1<<bit))
    mssg = f"bit {bit} of {val:08b} is {'clear' if skip else 'set'}"
    if skip:
        instr.skip()
    return [
        setreg(reg, val),
        SBRC(r(reg), str(bit)),
        instr(mssg)
    ]

In [8]:
sbrsTests = [
    "\n;PREPROCESS TestSBRS",
    LDI(r(31), v(0x00)),
    LDI(r(30), v(0x00)),
    "; skip a LD (1 word) instruction",
    testSbrc(choice(range(30)), 0xF0, 4, LD(r(0), 'Z').read(0, 0)),
    "; don't skip a LD instruction",
    testSbrc(choice(range(30)), 0x0F, 7, LD(r(0), 'Z').read(0, 0)),
    "; skip a STS (2 word) instruction",
    testSbrc(choice(range(32)), 0xF0, 5, checkreg(0,0xff)),
    "; don't skip a STS instruction",
    testSbrc(choice(range(32)), 0x0F, 6, checkreg(0,0xff)),
]

In [9]:
bits = [randint(0,1) == 0 for _ in range(8)]
vals = [randint(0, 0xFF) for _ in range(32)]
nopTests = [
    "\n;PREPROCESS TestNOP",
    "; set some random values in registers",
    [setstatus(i, bits[i]) for i in range(8)],
    [setreg(i, vals[i]) for i in range(32)],
    "; do a whole lot of nothing",
    NOP(),NOP(),NOP(),
    "; check that indeed nothing happened",
    [checkreg(i, vals[i]) for i in range(32)],
    [checkstatus(i, bits[i]) for i in range(8)],
    NOP()["done"]
]

In [10]:
skipTests = [
    "; Tests the flow control skip instructions and NOP\n",
    cpseTests,
    sbrcTests,
    sbrsTests,
    nopTests
]
with open("flow_skip.asm", "w") as f:
    f.write(stringify(skipTests))